In [1]:
# mpu6050_temp.py
import tdwf
import time
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Registers (integers)
PWR_MGMT1 = 0x6B
PWR_MGMT2 = 0x6C
WHO_AM_I = 0x75
SMPLRT_DIV = 0x19
CONFIG = 0x1A
TEMP_OUT_H = 0x41
TEMP_OUT_L = 0x42
SAD = 0x68

def twos_comp(val, bits=16):
    if val & (1 << (bits - 1)):
        val -= 1 << bits
    return val

def read_temperature(sht):
    # burst read 2 bytes from TEMP_OUT_H
    sht.writeread([TEMP_OUT_H], 2)  # read TEMP_OUT_H and TEMP_OUT_L
    high = sht.vals[0]
    low = sht.vals[1]
    raw = (high << 8) | low
    raw = twos_comp(raw, 16)
    # conversion: Temp (°C) = raw/340 + 36.53
    temp_c = raw / 340.0 + 36.53
    return temp_c, raw

def main():
    ad2 = tdwf.AD2()
    try:
        ad2.vdd = 3.3
        ad2.power(True)
        i2c = tdwf.I2Cbus(ad2.hdwf)
        devs = i2c.scan()
        print("I2C devices:", [hex(d) for d in devs])

        sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

        # Reset and basic configuration (wake up + enable sensors)
        sht.write([PWR_MGMT1, 0x80])  # device reset
        time.sleep(0.5)
        sht.write([PWR_MGMT1, 0x00])  # wake, clock default
        time.sleep(0.05)
        sht.write([PWR_MGMT2, 0x00])  # enable accel & gyro (temperature read unaffected by disabling)
        time.sleep(0.05)
        sht.write([CONFIG, 0x03])      # DLPF config (optional)
        time.sleep(0.01)
        sht.write([SMPLRT_DIV, 0x04])  # sample rate divider
        time.sleep(0.01)

        # Verify WHO_AM_I
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("WHO_AM_I mismatch: 0x%02x" % sht.vals[0])
            return

        # Setup plotting
        fig, ax = plt.subplots(figsize=(8,4))
        times = []
        temps = []
        start = time.time()

        running = True
        print("Press ESC in the plot window to stop.")
        def on_key(event):
            nonlocal running
            if event.key == 'escape':
                running = False
        fig.canvas.mpl_connect("key_press_event", on_key)

        while running:
            t, raw = read_temperature(sht)
            now = time.time() - start
            times.append(now)
            temps.append(t)
            print(f"{now:6.2f}s  T = {t:.2f} °C  (raw={raw})")

            ax.clear()
            ax.plot(times, temps, marker='o')
            ax.set_xlabel("Time [s]")
            ax.set_ylabel("Temperature [°C]")
            ax.grid(True)
            plt.pause(0.1)  # allows GUI event loop to run

            time.sleep(0.2)  # sample ~2-3 Hz, adjust as desired

    finally:
        print("Cleaning up hardware...")
        try:
            plt.close('all')
        except:
            pass
        ad2.power(False)
        ad2.close()
        print("AD2 Power Off. Process safely terminated.")

if __name__ == "__main__":
    main()


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
I2C devices: ['0x68']
Press ESC in the plot window to stop.
  0.00s  T = 29.78 °C  (raw=-2296)
  0.48s  T = 29.81 °C  (raw=-2286)
  0.81s  T = 29.84 °C  (raw=-2275)
  1.13s  T = 29.86 °C  (raw=-2267)
  1.45s  T = 29.89 °C  (raw=-2257)
  1.77s  T = 29.91 °C  (raw=-2251)
  2.16s  T = 29.89 °C  (raw=-2256)
  2.48s  T = 29.94 °C  (raw=-2242)
  2.80s  T = 29.97 °C  (raw=-2232)
  3.13s  T = 29.96 °C  (raw=-2235)
  3.46s  T = 29.97 °C  (raw=-2229)
  3.80s  T = 29.96 °C  (raw=-2233)
  4.13s  T = 29.96 °C  (raw=-2235)
  4.45s  T = 29.95 °C  (raw=-2236)
  4.79s  T = 29.93 °C  (raw=-2243)
  5.12s  T = 29.95 °C  (raw=-2238)
  5.44s  T = 29.93 °C  (raw=-2244)
  5.79s  T = 29.93 °C  (raw=-2245)
  6.12s  T = 29.92 °C  (raw=-2246)
  6.46s  T = 29.95 °C  (raw=-2238)
  6.79s  T = 29.94 °C  (raw=-2240)
  7.12s  T = 29.93 °C  (raw=-2243)
  7.46s  T = 29.93 °C  (raw=-2244)
  7.79s  

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\aless\miniconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\aless\miniconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "c:\Users\aless\miniconda3\Lib\site-packages\matplotlib\backends\_backend_tk.py", line 274, in idle_draw
    self.draw()
  File "c:\Users\aless\miniconda3\Lib\site-packages\matplotlib\backends\backend_tkagg.py", line 10, in draw
    super().draw()
  File "c:\Users\aless\miniconda3\Lib\site-packages\matplotlib\backends\backend_agg.py", line 388, in draw
    self.figure.draw(self.renderer)
  File "c:\Users\aless\miniconda3\Lib\site-packages\matplotlib\artist.py", line 95, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aless\miniconda3\Lib\site-packages\matplotlib\artist.py", line 72, in draw_wrap

347.40s  T = 28.47 °C  (raw=-2742)
347.74s  T = 28.46 °C  (raw=-2744)
348.05s  T = 28.46 °C  (raw=-2743)
348.37s  T = 28.47 °C  (raw=-2742)
348.69s  T = 28.46 °C  (raw=-2744)
349.09s  T = 28.48 °C  (raw=-2738)
349.41s  T = 28.46 °C  (raw=-2743)
349.76s  T = 28.48 °C  (raw=-2738)
350.08s  T = 28.46 °C  (raw=-2743)
350.40s  T = 28.49 °C  (raw=-2733)
350.72s  T = 28.48 °C  (raw=-2737)
351.05s  T = 28.49 °C  (raw=-2734)
351.38s  T = 28.49 °C  (raw=-2733)
351.70s  T = 28.51 °C  (raw=-2727)
352.02s  T = 28.52 °C  (raw=-2725)
352.40s  T = 28.52 °C  (raw=-2723)
352.72s  T = 28.52 °C  (raw=-2723)
353.08s  T = 28.57 °C  (raw=-2707)
353.40s  T = 28.59 °C  (raw=-2699)
353.72s  T = 28.61 °C  (raw=-2694)
354.05s  T = 28.58 °C  (raw=-2702)
354.36s  T = 28.61 °C  (raw=-2694)
354.68s  T = 28.64 °C  (raw=-2682)
355.00s  T = 28.63 °C  (raw=-2687)
355.31s  T = 28.67 °C  (raw=-2671)
355.63s  T = 28.66 °C  (raw=-2676)
355.95s  T = 28.68 °C  (raw=-2668)
356.27s  T = 28.69 °C  (raw=-2665)
356.60s  T = 28.71 °

KeyboardInterrupt: 